In [1]:
library(Seurat)
library(SetupCellLayers)

Attaching SeuratObject



In [2]:
sobj <- readRDS('../Data/PBMC/pbmc3k.rds')

In [3]:
Idents(sobj) <- 'seurat_annotations'
sobj <- subset(x = sobj, downsample = 100, subset=seurat_annotations!="<NA>")

In [4]:
# Basic Seurat analysis
sobj[["percent.mt"]] <- PercentageFeatureSet(sobj, pattern = "^MT-")
sobj <- subset(sobj, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)
sobj <- NormalizeData(sobj, normalization.method = "LogNormalize", scale.factor = 10000)
sobj <- FindVariableFeatures(sobj, selection.method = "vst", nfeatures = 2000)
all.genes <- rownames(sobj)
sobj <- ScaleData(sobj, features = all.genes)
sobj <- RunPCA(sobj, features = VariableFeatures(object = sobj))
sobj <- JackStraw(sobj, num.replicate = 100)
sobj <- ScoreJackStraw(sobj, dims = 1:20)
sobj <- FindNeighbors(sobj, dims = 1:10)
sobj <- RunUMAP(sobj, dims = 1:10)

Centering and scaling data matrix

PC_ 1 
Positive:  IL32, CD3D, CD3E, CTSW, GZMA, CST7, LDHB, CD247, LTB, NKG7 
	   STK17A, TPT1, CD2, CCL5, ACAP1, HOPX, PRF1, FGFBP2, ALOX5AP, GZMB 
	   NCR3, SELL, GZMH, GIMAP5, GNLY, CCL4, GZMK, KLRG1, ARPC5L, AQP3 
Negative:  CST3, LST1, AIF1, FTH1, FTL, CFD, TYMP, SERPINA1, CD68, IFITM3 
	   COTL1, LYZ, TYROBP, FCER1G, CTSS, FCN1, SAT1, SPI1, CFP, PSAP 
	   IFI30, LGALS1, S100A9, NPC2, S100A11, RP11-290F20.3, MS4A7, TIMP1, OAZ1, LGALS3 
PC_ 2 
Positive:  SDPR, GNG11, PF4, GP9, SPARC, PTCRA, TUBB1, PPBP, ITGA2B, TMEM40 
	   HIST1H2AC, MMD, AP001189.4, NRGN, CLU, CD9, ACRBP, RGS18, CA2, TSC22D1 
	   TREML1, F13A1, LY6G6F, GP1BA, CMTM5, C2orf88, SEPT5, CLEC1B, MYL9, MAP3K7CL 
Negative:  RPS2, TMSB10, S100A4, COX4I1, TPT1, S100A6, JUNB, S100A11, S100A10, FOS 
	   TYROBP, ZFP36, VIM, HLA-DPA1, LGALS1, CTSS, LST1, RBM3, IFITM2, CD48 
	   RAC2, TYMP, PYCARD, IFI6, FCN1, EMP3, C1orf162, IFITM3, AIF1, IFI30 
PC_ 3 
Positive:  CD79A, MS4A1, HLA-DRA, HLA-DQA

In [7]:
cl.setup <- compute_modularity(sobj, seq(0.1, 0.5, by=0.1),
                               '../Data/Test-SetupCellLayers/PBMC_modularity.csv')

In [8]:
cl.setup[['sil']] <- compute_silhouette_scores(cl.setup$sobj,
                                               as.character(cl.setup$mod$'resolution'),
                                               '../Data/Test-SetupCellLayers/PBMC_silhouette_scores.csv')